In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'C:/Users/Sahil Rafaliya/Downloads/archive/brain_tumor_dataset',
    target_size=(224, 224),  # Resize images to 150x150
    batch_size=32,
    class_mode='binary',  # Since you have two classes: yes and no
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'C:/Users/Sahil Rafaliya/Downloads/archive/brain_tumor_dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

Epoch 1/20


C:\Users\Sahil Rafaliya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.5964 - loss: 0.7506 - val_accuracy: 0.7812 - val_loss: 0.5172
Epoch 2/20
1/6 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.7188 - loss: 0.5879

C:\Users\Sahil Rafaliya\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 571ms/step - accuracy: 0.7188 - loss: 0.5879 - val_accuracy: 0.6667 - val_loss: 0.5446
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 32s 5s/step - accuracy: 0.7847 - loss: 0.4844
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.2311 - val_accuracy: 0.7500 - val_loss: 0.4701
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 30s 5s/step - accuracy: 0.8205 - loss: 0.3952 - val_accuracy: 0.7778 - val_loss: 0.4532
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9062 - loss: 0.2842
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 6s/step - accuracy: 0.8634 - loss: 0.3249 - val_accuracy: 0.7500 - val_loss: 0.5410
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 556ms/step - accuracy: 0.7812 - loss: 0.4570 - val_accuracy: 0.7778 - val_loss: 0.3376
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - accuracy: 0.8444 - loss: 0.3253
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8750 - loss: 0.2652 - val_accuracy: 0.7812 - val_loss: 0.4945
Epoch 11/20
6/6 ━

In [23]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = r"C:\Users\Sahil Rafaliya\Downloads\test brain.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Tumor Detected (Yes)")
else:
    print("No Tumor Detected (No)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
No Tumor Detected (No)


In [25]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.8279 - loss: 0.3559
Validation Loss: 0.36495712399482727
Validation Accuracy: 0.8199999928474426


In [27]:
# Unfreeze some layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train again
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.9811 - loss: 0.1280 - val_accuracy: 0.9062 - val_loss: 0.2814
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 373ms/step - accuracy: 0.9375 - loss: 0.2094 - val_accuracy: 0.7222 - val_loss: 0.4881
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.9615 - loss: 0.1166
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 788ms/step - accuracy: 0.9375 - loss: 0.1260 - val_accuracy: 0.8125 - val_loss: 0.4852
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 0.9950 - loss: 0.0684 - val_accuracy: 0.7778 - val_loss: 0.4411
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 1.0000 - loss: 0.0317
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 0.9848 - loss: 0.0895 - val_accuracy: 0.7812 - val_loss: 0.3645
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 438ms/step - accuracy: 1.0000 - loss: 0.0562 - val_accuracy: 0.8889 - val_loss: 0.4431
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 24s 4s/step - accuracy: 1.0000 - loss: 0.0465
Epoc

In [13]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Path to your dataset
data_path = "C:/Users/Sahil Rafaliya/Downloads/archive/brain_tumor_dataset/"

# Parameters
IMG_SIZE = (128, 128)  # Resize images to 128x128
BATCH_SIZE = 32

# Step 1: Load and preprocess images
datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values to [0, 1]
data_generator = datagen.flow_from_directory(
    data_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification: "yes" or "no"
    shuffle=False  # Keep order for mapping filenames to features
)

# Step 2: Use a pre-trained CNN (VGG16) to extract features
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Extract features for all images
features = vgg16.predict(data_generator)
features = features.reshape(features.shape[0], -1)  # Flatten features

# Get labels
labels = data_generator.classes  # Binary labels: 0 for "no", 1 for "yes"
class_names = list(data_generator.class_indices.keys())  # Map classes to names

# Step 3: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Step 4: Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))


Found 253 images belonging to 2 classes.


C:\Users\Sahil Rafaliya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
Accuracy: 0.9019607843137255

Classification Report:
              precision    recall  f1-score   support

          no       1.00      0.75      0.86        20
         yes       0.86      1.00      0.93        31

    accuracy                           0.90        51
   macro avg       0.93      0.88      0.89        51
weighted avg       0.92      0.90      0.90        51



In [11]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from sklearn.ensemble import RandomForestClassifier
import joblib  # For saving/loading models

# Step 1: Load the Random Forest model (if already trained and saved)
# rf_model = joblib.load('random_forest_model.pkl')  # Uncomment if you have saved the model
# If you're running the training code above, you can skip this step

# Step 2: Load and preprocess the test image
def preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)  # Load and resize the image
    image = img_to_array(image)  # Convert to a NumPy array
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Path to your test image
test_image_path = "C:/Users/Sahil Rafaliya/Downloads/archive/brain_tumor_dataset/yes/Y10.jpg"  # Replace with the actual path

# Preprocess the test image
preprocessed_image = preprocess_image(test_image_path)

# Step 3: Use VGG16 to extract features
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
test_features = vgg16.predict(preprocessed_image)
test_features = test_features.reshape(1, -1)  # Flatten features

# Step 4: Make a prediction with the Random Forest model
predicted_class = rf_model.predict(test_features)[0]

# Map the prediction to the class name
class_names = ["no", "yes"]  # 0: "no", 1: "yes"
predicted_label = class_names[int(predicted_class)]

print(f"Predicted Class: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step


NameError: name 'rf_model' is not defined

In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [15]:
X_train, X_test, y_train, y_test 

(array([[0.        , 0.        , 0.14652753, ..., 0.        , 0.79908717,
         0.        ],
        [0.        , 0.        , 0.19094937, ..., 0.        , 0.7463138 ,
         0.        ],
        [0.20291875, 0.        , 0.        , ..., 0.        , 0.7534152 ,
         0.        ],
        ...,
        [0.06687389, 0.        , 0.74761796, ..., 0.        , 0.9028589 ,
         0.        ],
        [0.06313635, 0.        , 0.        , ..., 0.        , 0.86433494,
         0.        ],
        [0.00553559, 0.        , 0.17625122, ..., 0.        , 0.8212719 ,
         0.        ]], dtype=float32),
 array([[0.        , 0.        , 0.08295961, ..., 0.        , 0.759027  ,
         0.        ],
        [0.01399012, 0.        , 0.2685593 , ..., 0.        , 0.49399891,
         0.        ],
        [0.0753444 , 0.        , 0.        , ..., 0.        , 0.9191039 ,
         0.        ],
        ...,
        [0.        , 0.        , 0.02906484, ..., 0.        , 0.7294759 ,
         0.        